In [1]:
!pip install gensim
!pip install nltk
!pip install pyspark
!pip install pyarrow

You should consider upgrading via the 'c:\users\aniru\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\aniru\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\aniru\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import numpy as np
import pandas as pd
import os
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from gensim import corpora, models
from pprint import pprint
from tqdm import tqdm
from pyspark.sql import SQLContext
from pyspark.sql.functions import isnan, when, count, col, translate
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql import functions as F
import ast 

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
os.chdir('..')

In [3]:
!pwd

C:\Users\aniru\Desktop\Amrita\Sem 5\Big Data and Database Management\Project\Kisan-Query-Analysis


In [4]:
from pyspark import SparkContext
sc = SparkContext("local", "Kisan_Query_Answering")
sqlContext = SQLContext(sc)

c:\users\aniru\appdata\local\programs\python\python39\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Functions

In [5]:
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in tqdm(enumerate(ldamodel[corpus])):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

## Load required variables

In [6]:
dictionary_qt = pickle.load(open("models\\dictionary", "rb"))
tfidf_qt = pickle.load(open("models\\tfidf", "rb"))
lda_model_qt_tfidf = pickle.load(open("models\\lda_model_tfidf", "rb"))
# corpus_qt_tfidf = pickle.load(open("models\\corpus_tfidf", "rb"))
corp = pickle.load(open("models\\corp_tfidf", "rb"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
df_dominant_topic = sqlContext.read.csv("models\\Dominant_Topic_for_Queries.csv", header=True, inferSchema = True)
print(df_dominant_topic.count())

data = sqlContext.read.csv("Data\\query_agg.csv", header=True, inferSchema = True)
print(data.count())

213396
7072487


In [8]:
data.show(5)

+------+------------+----------+--------+---------+--------------------+--------------------+---------+------------+---------+-------------------+
|Season|      Sector|  Category|    Crop|QueryType|           QueryText|              KccAns|StateName|DistrictName|BlockName|          CreatedOn|
+------+------------+----------+--------+---------+--------------------+--------------------+---------+------------+---------+-------------------+
|  null| AGRICULTURE|   Cereals|   Wheat|        0|INFORMATION REGAR...|SPRAY OF TILT(PRO...|   PUNJAB|    AMRITSAR| AJNALA-1|2013-04-01T06:51:45|
|  null|HORTICULTURE|Vegetables|Chillies|        0|INFORMATION REGAR...|THE PHALARIS MINO...|   PUNJAB|    AMRITSAR| AJNALA-1|2013-04-01T07:29:19|
|  RABI|HORTICULTURE|Vegetables|    1282|       76|control of worm i...|                null|   PUNJAB|    AMRITSAR| AJNALA-1|2013-04-01T07:35:38|
|KHARIF| AGRICULTURE|   Cereals|   Wheat|        3|INFORMATION REGAR...|SPRAY OF EKALUX[Q...|   PUNJAB|    AMRITSAR| A

In [9]:
df_dominant_topic.show(5)

+-----------+--------------+------------------+--------------------+----+
|Document_No|Dominant_Topic|Topic_Perc_Contrib|            Keywords|Text|
+-----------+--------------+------------------+--------------------+----+
|          0|           3.0|            0.3249|report, fertil, d...|null|
|          1|           2.0|             0.473|forecast, weather...|null|
|          2|           2.0|            0.4605|forecast, weather...|null|
|          3|           6.0|             0.703|market, ask, pest...|null|
|          4|           4.0|            0.4585|control, leaf, br...|null|
+-----------+--------------+------------------+--------------------+----+
only showing top 5 rows



In [10]:
df_dominant_topic = df_dominant_topic.where(col("Text").isNotNull())

In [11]:
df_dominant_topic.show(10)

+-----------+--------------+------------------+--------------------+--------------------+
|Document_No|Dominant_Topic|Topic_Perc_Contrib|            Keywords|                Text|
+-----------+--------------+------------------+--------------------+--------------------+
|          7|           0.0|            0.6363|weather, inform, ...|asking about weat...|
|         13|           3.0|            0.5801|report, fertil, d...|new varieties of ...|
|         48|           6.0|            0.6208|market, ask, pest...|#   INFORMATION R...|
|         72|           6.0|            0.5746|market, ask, pest...|WHETHER TO NEED I...|
|         80|           6.0|            0.7317|market, ask, pest...|control of stem r...|
|         89|           3.0|            0.3331|report, fertil, d...|INFORMATION REGAR...|
|        105|           3.0|            0.6571|report, fertil, d...|INFORMATION REGAR...|
|        135|           1.0|            0.4174|tell, varieti, nu...|information regar...|
|        1

In [12]:
data.show(10)

+------+------------+----------+--------+---------+--------------------+--------------------+---------+------------+---------+-------------------+
|Season|      Sector|  Category|    Crop|QueryType|           QueryText|              KccAns|StateName|DistrictName|BlockName|          CreatedOn|
+------+------------+----------+--------+---------+--------------------+--------------------+---------+------------+---------+-------------------+
|  null| AGRICULTURE|   Cereals|   Wheat|        0|INFORMATION REGAR...|SPRAY OF TILT(PRO...|   PUNJAB|    AMRITSAR| AJNALA-1|2013-04-01T06:51:45|
|  null|HORTICULTURE|Vegetables|Chillies|        0|INFORMATION REGAR...|THE PHALARIS MINO...|   PUNJAB|    AMRITSAR| AJNALA-1|2013-04-01T07:29:19|
|  RABI|HORTICULTURE|Vegetables|    1282|       76|control of worm i...|                null|   PUNJAB|    AMRITSAR| AJNALA-1|2013-04-01T07:35:38|
|KHARIF| AGRICULTURE|   Cereals|   Wheat|        3|INFORMATION REGAR...|SPRAY OF EKALUX[Q...|   PUNJAB|    AMRITSAR| A

## Pipeline:

In [19]:
def pipeline(tt):
    tt = pd.Series(tt)
    processed = tt.map(preprocess)
#     dictionary = gensim.corpora.Dictionary(processed)
    bow_corpus = [dictionary_qt.doc2bow(doc) for doc in processed]
#     tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf_qt[bow_corpus]
#     print("Processed: {} \n BoW: {} \n Corpus TF-IDF: {}".format(processed, bow_corpus, corpus_tfidf[0]))
#     ctr = True
#     for index, score in sorted(lda_model_qt_tfidf[bow_corpus[0]], key=lambda tup: -1*tup[1]):
#         print("\nScore: {}\t \nTopic: {}".format(score, lda_model_qt_tfidf.print_topic(index, 8)))
    
    tops = lda_model_qt_tfidf.get_document_topics(corpus_tfidf[0])
    max1 = (0,0)
    for i in tops:
        if(i[1] > max1[1]):
            max1 = i
    aa1 = df_dominant_topic.where(col("Text").isNotNull())
    pos_doc = aa1.where(col("Dominant_Topic") == max1[0]).take(10)
    a1 = [i['Text'] for i in pos_doc]
    
    fin_ans = []
    for i in data1.values:
        if(i[5] in a1):
            fin_ans.append(i[6])
    return fin_ans[:10]

In [21]:
aa = "weather"
fin = pipeline(aa)
fin

['it may be rainfall on 2rd april',
 'Mostly dry. Warm (max 37°C on Tue afternoon, min 24°C on Sun night). Wind will be generally light.',
 'spray of 60 ml alfamethrin with 100 ltr water',
 'Some drizzle, heaviest during Thu morning. Warm (max 35°C on Wed afternoon, min 25°C on Tue night). Wind will be generally light.',
 '6 MARLE /8KG SEED',
 'Mostly dry. Warm (max 37°C on Tue afternoon, min 26°C on Sun night). Wind will be generally light.',
 'SOME CHANCES OF RAIN IN NEXT 24 HOURS',
 'SPRAY RIPCORD (10 EC) @ 200 ML IN 150 LITRE WATER/ACRE.',
 'SOME CHANCES OF RAIN IN NEXT 24 HOURS',
 'SOME CLOUDS AND NO CHANCES OF RAINFALL .']

In [22]:
aa = "pesticides"
fin = pipeline(aa)
fin

['SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SPRAY OF STOMP 30 EC @ 1 LITRE IN THE 200 LITRESOF WATER/ACRE AT PRE-EMERGENCE STAGE.',
 'THERE IS NO NEED TO IRRIGATE THE WHEAT.',
 'spray bavistin 250gm in 150lit water per acre.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'Light rain (total 2mm), mostly falling on  night',
 'SPRAY ROGOR/METASYSTOX @ 150 ML IN 100 LITERS OF WATER/ACRE.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115

In [23]:
aa = "paddy"
fin = pipeline(aa)
fin

['SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-115,PR-113,PUSA-44,HKR-47,PUSA PUNJAB BASMATI-1509,PUNJAB BASMATI-3.',
 'SUITABLE VARIETIES OF PADDY(JHONA)-PR-121,PR-122,PR-118,PR-114,PR-111,PR-116,PR-

## End

In [20]:
# Load the data(Variable) from the kaggle site
# df_dominant_topic = pd.read_csv('models\\Dominant_Topic_for_Queries.csv')
data1 = pd.read_csv('Data\\query_agg.csv')